<a href="https://colab.research.google.com/github/MatSpad/Advanced-Coding-for-Data-Analytics-Project-2024/blob/main/ACfDA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Coding for Data Analytics Project
30/04/24
### Group X:
Federico De Marinis (xxxxxx)

Eleonora Di Mario (xxxxxx)

Matteo Spadaccia (277141)

## Introduction
TODO: Intro to the topic and aims of the project

## Task 0 - Preliminary code
In this section, the contextual basis for the analysis is set.

### 0.1 - Sript settings
Some variables are settable to customize the specified script's behaviors.

In [39]:
# [Code: 1]

# Script settings:              > Usage:
reloadRepository = False        # if true, the GitHub repository is loaded again even if already cloned
rebuildGraph = True             # if true, the graph is built again from input data even if already available as saved object
loadDisconnectedNodes = True    # if true, when eventually rebuilding the graph, all the nodes are considered even if not partecipating in any edge
overwriteSavings = True         # if true, saved files are overwrited by newly computed outputs
generateImages = False          # if true, images are generated during code execution (computationally expensive)

# Checks:
repositoryLoaded = os.getcwd()[-17:] == 'cloned-repository' # checks if the current working directory is the cloned repository itself
if not rebuildGraph and os.path.isfile('Outputs/Graph.pickle'):
  rebuildGraph = True
  print("No previously saved graph object found, the network will be constructed again from data.")

print("Script set!")

Script set!


### 0.2 - Importing libraries and functions
Some libraries and functions are imported to ease the specified following tasks.

In [40]:
# [Code: 2]

# Importing libraries:          > Useful for:
import os                       # managing directories
import shutil                   # deleting directories
import time                     # checking code run times
from tqdm.auto import tqdm      # generating fancy progress bars
import gzip                     # decompressing and reading .gz files
import networkx as nx           # generating and analyzing graph objects
import pandas as pd             # managing dataframes
import pickle                   # storing objects (e.g., NetworkX graphs) as binary files
import matplotlib.pyplot as plt # managing image-format visual representations
import matplotlib.image as mpimg# reading image-format files

# Importing functions:          > Useful for:
def section(x:str):             # clearly depict sections of output (TO BE STORED SEPARATELY)
  if x.lower() == "done":
    print('_'*74+" DONE!\n\n")
  else:
    print(x+' '+'_'*(79-len(x)))

print("Libraries and functions imported!")

Libraries and functions imported!


### 0.3 - Colab-GitHub interaction initialization
Colab's working directory is set by loading the project's GitHub repository. If the latter has already been cloned and the reloadRepository variable [Code: 1 - Script settings] is switched to False, the working directory would not be updated.

In [41]:
# [Code: 3]

# Colab-GitHub interaction initialization
if repositoryLoaded and reloadRepository:                   # if the repository is already loaded and to be updated, it is discarded
  os.chdir('..')
  shutil.rmtree('cloned-repository')
  repositoryLoaded = False
if not repositoryLoaded:                                    # if the repository is to be loaded (or updated), it is cloned from GitHub
  section("Loading repository:")
  ! git clone -l -s https://github.com/MatSpad/Advanced-Coding-for-Data-Analytics-Project-2024.git cloned-repository
  %cd cloned-repository
  section("Done")
else:
  print("Using previously loaded repository!")

Using previously loaded repository!


## Task 1 - Constructing the biomedical knowledge graph
In this section, the nodes and edges data are imported and saved as a NetworkX directed multi-graph, on which specific analysis could be run afterwards.

### 1.1 - Building Dataframes from files
The information contained in both the nodelist and edgelist files are loaded as separate Pandas dataframes.

In [42]:
# [Code: 4]

# Bulding nodes dataframe from the related file
section("Loading nodes' info as dataframe:")
nodesDF = pd.read_csv('Inputs/hetionet-v1.0-nodes.tsv',sep='\t')
display(nodesDF)
section("Done")

# Bulding edges dataframe from the related file
section("Loading edges list as dataframe:")
edgesDF = pd.read_csv('Inputs/hetionet-v1.0-edges.sif.gz', compression='gzip', sep='\t')
display(edgesDF)
section("Done")

Loading nodes' info as dataframe: ______________________________________________


,id,name,kind
0,Anatomy::UBERON:0000002,uterine cervix,Anatomy
1,Anatomy::UBERON:0000004,nose,Anatomy
2,Anatomy::UBERON:0000006,islet of Langerhans,Anatomy
3,Anatomy::UBERON:0000007,pituitary gland,Anatomy
4,Anatomy::UBERON:0000010,peripheral nervous system,Anatomy
...,...,...,...
47026,Symptom::D064250,Hypertriglyceridemic Waist,Symptom
47027,Symptom::D065634,Cerebrospinal Fluid Leak,Symptom
47028,Symptom::D065635,Benign Paroxysmal Positional Vertigo,Symptom
47029,Symptom::D065906,Hyperlactatemia,Symptom


__________________________________________________________________________ DONE!


Loading edges list as dataframe: _______________________________________________


,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898
...,...,...,...
2250192,Anatomy::UBERON:0000057,AeG,Gene::65009
2250193,Anatomy::UBERON:0000474,AeG,Gene::80279
2250194,Anatomy::UBERON:0002048,AeG,Gene::1211
2250195,Anatomy::UBERON:0002048,AeG,Gene::8843


__________________________________________________________________________ DONE!




### 1.2 - Preliminary analysis
Some preliminary checks are enacted on the dataframes in order to determine which type of graph could better store the given data (i.e., a directed multigraph).

In [43]:
# [Code: 5]

# Preliminary analysis:                                                                                                                           > Finding:
nodesFileLen = len(nodesDF.index)
uniqueNodeIDs = len(pd.unique(nodesDF['id']))

edgesFileLen = len(edgesDF.index)
source_targetCouples = len(edgesDF.loc[:, ['source', 'target']].drop_duplicates())
uniqueEdges = len(edgesDF.drop_duplicates())

print("Nodes in DF:", nodesFileLen, "(of which", uniqueNodeIDs, "different by ID)")                                                               # the nodes data present 47031 different nodes
print("Edges in DF:", edgesFileLen, "(of which", uniqueEdges, "different, but just", source_targetCouples, "with different source-target pair)")  # the edges data present 2250197 edges, some with equal source-target pair but different link type

Nodes in DF: 47031 (of which 47031 different by ID)
Edges in DF: 2250197 (of which 2250197 different, but just 2110272 with different source-target pair)


### 1.3 - Constructing the MultiDiGraph
A NetworkX directed multi-graph containing all the nodes' and edges' information is built - and eventually saved overwriting the previous binary file version - or extracted as object from a previously saved file.

In [44]:
# [Code: 6]

if rebuildGraph:

  # Constructing the MultiDiGRaph by reading the edges file directly (with progress bar)
  '''
  start = time.time()

  section("Loading the graph blending both nodes' and edges' data:")
  G = nx.MultiDiGraph()
  with gzip.open('Inputs/hetionet-v1.0-edges.sif.gz','r') as edgesFile:
    edgesFile.readline()
    for i in tqdm(range(edgesFileLen)):
      lineElements = str(edgesFile.readline())[2:-5].split('\\t')
      G.add_edge(lineElements[0], lineElements[2], edgeType=lineElements[1])
  edgesNum = G.number_of_edges()
  nodesNum = G.number_of_nodes()
  print(edgesNum, nodesNum)
  section("Done")

  end = time.time()
  print(end - start)  # progress-bar-enriched graph building is slightly slower than plain one
  '''

  # Constructing the MultiDiGRaph by reading both the dataframes
  G = nx.from_pandas_edgelist(edgesDF, 'source', 'target', edge_attr='metaedge', create_using=nx.MultiDiGraph())
  edgesNum = G.number_of_edges()
  connectedNodesNum = G.number_of_nodes()

  # Eventually adding disconnected nodes too
  if loadDisconnectedNodes:
    G.add_nodes_from(nodesDF['id'])
  nodesNum = G.number_of_nodes()

  # Setting all nodes' attributes
  nx.set_node_attributes(G, nodesDF.set_index('id').to_dict('index'), 'attributes')

  # Saving the graph object to file
  if overwriteSavings:
    pickle.dump(G, open('Outputs/Graph.pickle', 'wb'))
    print("MultiDiGraph built and saved!")
  else:
    print("MultiDiGraph built!")

else:
  # Loading the graph object from file
  G = pickle.load(open('Outputs/Graph.pickle', 'rb'))
  print("MultiDiGraph loaded from previously saved file!")

# Displaying the loaded graph's info
if not rebuildGraph:
  plt.imshow(mpimg.imread('Outputs/Graph.png'))
  plt.show()
elif generateImages:
  nx.draw_kamada_kawai(G)
  if overwriteSavings:
    plt.savefig('Outputs/Graph.png', dpi=300, bbox_inches='tight')
  plt.show()
print("Number of edges:", edgesNum, "\nNumber of nodes:", nodesNum)
if loadDisconnectedNodes or not rebuildGraph:
  print("(of which", connectedNodesNum, "connected)")

MultiDiGraph built and saved!
Number of edges: 2250197 
Number of nodes: 47031
(of which 45158 connected)


## Task 2 – Analyzing the biomedical knowledge graph:
In this section, TODO...

In [45]:
# USEFUL AFTERWARDS:

# Checking a node's attributes given its ID
nx.get_node_attributes(G, 'attributes')['Anatomy::UBERON:0000002']

{'name': 'uterine cervix', 'kind': 'Anatomy'}